<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-description" data-toc-modified-id="Problem-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem description</a></span><ul class="toc-item"><li><span><a href="#Programming-exercise" data-toc-modified-id="Programming-exercise-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Programming exercise</a></span></li></ul></li></ul></div>

# Problem description

Discrete time convolution (a DSP term) of an input signal $x[n]$ and a filter $h[n]$ is defined in textbooks as:
$$
y = h * x = x * h
\implies
y[m]
= \sum_{k=-\infty}^{\infty} h[k] x[m-k]
= \sum_{n=-\infty}^{\infty} x[n] h[m-n]
,$$
where the "$*$" above denotes convolution, not ordinary multiplication.

In practice we often use finite-length signals and filters, and not all textbooks define this case clearly.
The convolution of an input signal $(x[0],\ldots,x[N-1])$ with a finite impulse response (FIR) filter $(h[0], \ldots, h[K-1])$ becomes:
$$
y[m]
= \sum_{k=0}^{\min(K-1,m)} h[k] x[m-k]
= \sum_{n=0}^{\min(N-1,m)} x[n] h[m-n]
,\quad
m=0,\ldots,M-1
.$$
This is exactly the type of convolution used in "convolutional neural networks" that are a hot topic in machine learning.

Determine $M$, the length of the (possibly) non-zero part of $y$, in terms of $N$ and $K$.
We will use the notation $y,h,x$ to denote the finite vectors of length $M$, $K$, and $N$, respectively.

Convolution is a linear operation, so we can express it as a matrix-vector operation of the form $y = H x$ where $H$ is a matrix that you must determine and implement in this problem.
With some recycling of notation, you must implement $H$ such that $y = h * x = H x$.

**Caution:**
DSP notation uses signals that start at $n=0$, whereas the first element of a vector in `Julia` (and `Matlab`) is indexed by 1.
This is something you must get used to handling properly. 

**Hint:** It may be helpful to think through carefully the most reasonable size of $H$ first. 
You should not augment $x$, nor have any rows that are identically zero in $H$.

Write a function called `convolution` that takes as its input the vectors $h$ and $x$ and returns as output both the matrix $H$ and the column vector $y$ (the convolved signal, implemented without `conv`).

**Hint:** Compare your answer with the builtin convolution function: `Julia`'s `conv` function. Here is an example.


In [1]:
import Pkg; Pkg.add("DSP")
using DSP
conv([1; 2],[3; 4; 5])

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %13.6 %>                             ]  27.3 %                  ]  54.5 %                ]  58.8 %=============================>           ]  72.4 %]  85.9 % [========================================>]  99.6 % Resolving package versions...
 Installed Combinatorics ───── v1.0.0
 Installed TimerOutputs ────── v0.5.3
 Installed PlotUtils ───────── v0.6.0
 Installed IterTools ───────── v1.3.0
 Installed HTTP ────────────── v0.8.8
 Installed NearestNeighbors ── v0.4.4
 Installed CategoricalArrays ─ v0.7.3
 Installed DSP ─────────────── v0.6.0
 Installed Parsers ─────────── v0.3.10
 Installed ArrayInterface ──── v2.0.0
 Installed NaNMath ─────────── v0.3.3
 Installed TimeZones ───────── v0.10.3
 Installed PlotThemes ──────── v1.0.0
 Installed DiffEqDiffTools ─── v1.5.0
 Installed DataStructures ──── v0.17.6


┌ Info: Precompiling DSP [717857b8-e6f2-59f4-9121-6e50c889abd2]
└ @ Base loading.jl:1186


4-element Array{Int64,1}:
  3
 10
 13
 10

In [3]:
conv([1; 2; 3],[4; 5; 6; 7])

6-element Array{Int64,1}:
  4
 13
 28
 34
 32
 21

In [5]:
using SparseArrays, LinearAlgebra

In [7]:
for i = 1:5
    println(i)
end

1
2
3
4
5


Using your function below you should get that
```julia
    
    H,x = convolution(h,x)
    @show H*x
    @show conv(h[:],x[:]) # should be equal 
```

##  Programming exercise

In [8]:

function convolution(h, x)
#
# Syntax:       H, y = convolution(h, x)
#
# Inputs:       h is a vector of length K
#
#               x is a vector of length N
#
# Outputs:      H is the (N + K - 1) x N convolution matrix defined by h
#
#               y is the vector of length (N + K - 1) containing the discrete
#               convolution of h and x, but *not* computed via conv() calls.
#
# Description:  Computes the discrete convolution of the input vectors via
#               matrix multiplication and returns both the matrix and y
#


h = h[:]
x = x[:]
K = length(h)
N = length(x)
H=zeros(N+K-1, N)

for i =1 : N
    for j = 1 : K
        H[i+j-1,i] = h[j]
    end
end
    
# Compute convolution via matrix multiplication
y = H * x

return H, y
    
end

convolution (generic function with 1 method)

In one-based-indexing languages such as `MATLAB` or `Julia`, we might have used the variation

$$
y[j] = \sum_{k=-\infty}^{\infty} h[k] x[j-k+1] = \sum_{k=-\infty}^{\infty} h[j-k+1]x[k] .
$$

The equality $\sum_{k=-\infty}^{\infty} h[k] x[j-k+1] = \sum_{k=-\infty}^{\infty}  h[j-k+1]x[k]$,  the commutativity property of convolution, can be understood by realizing that either way $y[j]$ is a sum of all $x$'s and $h$'s whose indices add to $j+1$.

Sometimes this is written
$$y[j] = \sum_{\alpha+\beta=j+1} x[\alpha] y[\beta],$$
to emphasize the symmetry.
Returning to the problem at hand, we would like an $H$ that expresses matrix times vector multiplication:
$$y_j = \sum_k  H_{jk} x_k = \sum_{k=1}^n h[j-k+1]x[k].$$
Thus it is easy to see that
$$H_{jk} = h[j-k+1],$$
if $1 \le j-k+1 \le m$ and 0 otherwise. The $H$ looks like this
$$
H = 
\begin{bmatrix}
h[1] & 0    &  0 & \ldots & 0 \\
h[2] & h[1] &  0 & \ldots & 0\\
h[3] & h[2] &  h[1] & \ldots & 0\\
\vdots & \vdots & \vdots & \vdots &  \vdots \\
0    & 0    & 0    & 0  & h[m] 
\end{bmatrix}.
$$

A possible `Julia` implementation is
```julia
function convolution(h, x)
#
# Syntax:       H, y = convolution(h, x)
#
# Inputs:       h is a vector of length K
#
#               x is a vector of length N
#
# Outputs:      H is the (N + K - 1) x n convolution matrix defined by h
#
#               y is the vector of length (N + K - 1) containing the discrete
#               convolution of h and x, but *not* computed via conv() calls.
#
# Description:  Computes the discrete convolution of the input vectors via
#               matrix multiplication and returns both the matrix and y
#

    # Parse inputs
    h = vec(h) # we're so nice...
    x = vec(x)
    K = length(h)
    N = length(x)

    # Construct convolution matrix
    H = zeros(N + K - 1, N)
    for n in 1:N
        H[n:(n + K - 1), n] = h
    end

    # Compute convolution via matrix multiplication
    y = H * x

    return H, y
end
```

In `Julia`, we can construct $H$ elegantly using the ternary operator `a ? b : c` and array comprehensions as
```julia
H = [(1 <= j-k+1 <= m) ? h[j-k+1] : 0.0 for j=1:(m+n-1), k=1:n]
```
which reads as "if $1 \le j-k+1 \le m$, set $H[j,k] = h[j-k+1]$, otherwise set $H[j,k] = 0$."

Thus an alternative *prettier* solution is
```julia
function convolution(h,x)
    m, n = length(h), length(x)
    H = [(1 <= j-k+1 <= m) ? h[j-k+1] : 0.0 for j=1:(m+n-1), k=1:n]
    return H, H * x
end
```